In [89]:
from bs4 import BeautifulSoup
import feedparser
import re
import pandas as pd
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.corpus import stopwords 
import string
from pymongo import MongoClient
import pyodbc
import json

In [90]:
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Lokesh
[nltk_data]     Grandhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [91]:
##https://www.ndtv.com/rss
ndtv_feeds = ['https://feeds.feedburner.com/ndtvnews-world-news','https://feeds.feedburner.com/ndtvnews-top-stories',
             'https://feeds.feedburner.com/ndtvprofit-latest','https://feeds.feedburner.com/carandbike-latest',
             'https://feeds.feedburner.com/gadgets360-latest']

##https://www.foxnews.com/story/foxnews-com-rss-feeds
fox_feeds = ['http://feeds.foxnews.com/foxnews/world','http://feeds.foxnews.com/foxnews/business',
             'http://feeds.foxnews.com/foxnews/scitech','http://feeds.foxnews.com/foxnews/health',
            'http://feeds.foxnews.com/foxnews/politics','http://feeds.foxnews.com/foxnews/entertainment',
             'http://feeds.foxnews.com/foxnews/national']

##https://archive.nytimes.com/www.nytimes.com/services/xml/rss/index.html
ny_feeds = ['https://rss.nytimes.com/services/xml/rss/nyt/World.xml','https://rss.nytimes.com/services/xml/rss/nyt/Science.xml'
             ,'https://rss.nytimes.com/services/xml/rss/nyt/Business.xml']

all_feeds = [ndtv_feeds,fox_feeds,ny_feeds]

col = ['source','title','summary','link','published']
data_df = pd.DataFrame(columns = col)

def clean_data(news):
    final = BeautifulSoup(news, "lxml").text
    return final

for i in range(len(all_feeds)):
    if i == 0:
        source = 'NDTV'
    elif i == 1:
        source = 'FOX'
    elif i == 2:
        source = 'New York Times'
    for news_item in all_feeds[i]:
        feed = feedparser.parse(news_item)
        for news in feed['items']:
            data_df = data_df.append({
                'source' : source,
                'title' : news['title'],
                'summary' : clean_data(news['summary']),
                'link' : news['link'],
                'published' : news['published']
            }, ignore_index=True)

In [92]:
data_df.groupby('source').count()##Finding count

,title,summary,link,published
source,,,,
FOX,90,90,90,90
NDTV,100,100,100,100
New York Times,161,161,161,161


In [93]:
data_df

,source,title,summary,link,published
0,NDTV,Amazon Restores Service After Global Outage,Amazon.com Inc said its online stores had retu...,https://www.ndtv.com/world-news/amazon-restore...,"Mon, 12 Jul 2021 11:05:46 +0530"
1,NDTV,Party Time: Champagne And Celebrities Mark Ric...,"Champagne flowed, guests cheered and Grammy-no...",https://www.ndtv.com/world-news/party-time-cha...,"Mon, 12 Jul 2021 10:23:27 +0530"
2,NDTV,"Sydney's Daily Delta Cases Spike In ""Worst Hea...",Sydney recorded 112 new coronavirus cases on M...,https://www.ndtv.com/world-news/sydneys-daily-...,"Mon, 12 Jul 2021 10:12:10 +0530"
3,NDTV,Woman Infected With 2 Variants Highlights Next...,A 90-year-old woman died after becoming infect...,https://www.ndtv.com/world-news/woman-infected...,"Mon, 12 Jul 2021 09:56:11 +0530"
4,NDTV,Australia's Delta Variant Outbreak Worsens Des...,Australia on Monday reported its biggest daily...,https://www.ndtv.com/world-news/sydney-braces-...,"Mon, 12 Jul 2021 08:20:21 +0530"
...,...,...,...,...,...
346,New York Times,European Central Bank Tweaks Strategy to Fight...,The central bank said it would tolerate inflat...,https://www.nytimes.com/2021/07/08/business/ec...,"Thu, 08 Jul 2021 18:18:48 +0000"
347,New York Times,Ireland Fights for Its Tax Breaks,It and others are pushing back on a global agr...,https://www.nytimes.com/2021/07/08/business/de...,"Sat, 10 Jul 2021 19:39:53 +0000"
348,New York Times,A Digital Cat Is Melting Hearts (and Napping a...,The calico prances and dozes on a 26-by-62-foo...,https://www.nytimes.com/2021/07/08/world/asia/...,"Thu, 08 Jul 2021 11:05:48 +0000"
349,New York Times,The Parking Garage of the Future,Entrepreneurs envision a “mobility hub” that c...,https://www.nytimes.com/2021/07/08/business/pa...,"Thu, 08 Jul 2021 13:15:42 +0000"


In [94]:
##Sentence scleaning

data_df['clean_summary'] = data_df['summary'].str.replace('[^\w\s]','') ##Removing punctuations
data_df['clean_summary'] = data_df['clean_summary'].str.replace('\d+', '') ##Removing numbers
data_df['clean_summary'] = data_df['clean_summary'].apply(lambda x: " ".join(x.lower() for x in x.split())) ##Lower casing

stop = stopwords.words('english')
data_df['cleaned'] = data_df['clean_summary'].apply(lambda Y: " ".join(Y for Y in Y.split() if Y not in stop)) ##Removing stop words

In [95]:
data_df[['summary','cleaned']]

,summary,cleaned
0,Amazon.com Inc said its online stores had retu...,amazoncom inc said online stores returned norm...
1,"Champagne flowed, guests cheered and Grammy-no...",champagne flowed guests cheered grammynominate...
2,Sydney recorded 112 new coronavirus cases on M...,sydney recorded new coronavirus cases monday s...
3,A 90-year-old woman died after becoming infect...,yearold woman died becoming infected two diffe...
4,Australia on Monday reported its biggest daily...,australia monday reported biggest daily rise c...
...,...,...
346,The central bank said it would tolerate inflat...,central bank said would tolerate inflation off...
347,It and others are pushing back on a global agr...,others pushing back global agreement make comp...
348,The calico prances and dozes on a 26-by-62-foo...,calico prances dozes byfoot led billboard toky...
349,Entrepreneurs envision a “mobility hub” that c...,entrepreneurs envision mobility hub save spot ...


In [96]:
##Finding sentiment score
def vander_sentiment(data):
    v_analyser = SentimentIntensityAnalyzer()
    out = v_analyser.polarity_scores(data)
    return out['compound']

##Spliting words
def toekn(data):
    tok = nltk.word_tokenize(data)
    return tok

In [97]:
##Finding Sentiment based on compound score
data_df['sentiment'] = data_df['cleaned'].apply(lambda X : 'Positive' if vander_sentiment(X)>=0.05 else 'Negative' if vander_sentiment(X)<=-0.5 else 'Neutral')

##List of words
data_df['words'] = data_df['cleaned'].apply(lambda X : toekn(X))

In [98]:
data_df[['cleaned','sentiment','words']]

,cleaned,sentiment,words
0,amazoncom inc said online stores returned norm...,Neutral,"[amazoncom, inc, said, online, stores, returne..."
1,champagne flowed guests cheered grammynominate...,Positive,"[champagne, flowed, guests, cheered, grammynom..."
2,sydney recorded new coronavirus cases monday s...,Negative,"[sydney, recorded, new, coronavirus, cases, mo..."
3,yearold woman died becoming infected two diffe...,Negative,"[yearold, woman, died, becoming, infected, two..."
4,australia monday reported biggest daily rise c...,Neutral,"[australia, monday, reported, biggest, daily, ..."
...,...,...,...
346,central bank said would tolerate inflation off...,Neutral,"[central, bank, said, would, tolerate, inflati..."
347,others pushing back global agreement make comp...,Positive,"[others, pushing, back, global, agreement, mak..."
348,calico prances dozes byfoot led billboard toky...,Positive,"[calico, prances, dozes, byfoot, led, billboar..."
349,entrepreneurs envision mobility hub save spot ...,Positive,"[entrepreneurs, envision, mobility, hub, save,..."


In [99]:
final_df = data_df[['source','title','summary','sentiment','words']]
final_df

,source,title,summary,sentiment,words
0,NDTV,Amazon Restores Service After Global Outage,Amazon.com Inc said its online stores had retu...,Neutral,"[amazoncom, inc, said, online, stores, returne..."
1,NDTV,Party Time: Champagne And Celebrities Mark Ric...,"Champagne flowed, guests cheered and Grammy-no...",Positive,"[champagne, flowed, guests, cheered, grammynom..."
2,NDTV,"Sydney's Daily Delta Cases Spike In ""Worst Hea...",Sydney recorded 112 new coronavirus cases on M...,Negative,"[sydney, recorded, new, coronavirus, cases, mo..."
3,NDTV,Woman Infected With 2 Variants Highlights Next...,A 90-year-old woman died after becoming infect...,Negative,"[yearold, woman, died, becoming, infected, two..."
4,NDTV,Australia's Delta Variant Outbreak Worsens Des...,Australia on Monday reported its biggest daily...,Neutral,"[australia, monday, reported, biggest, daily, ..."
...,...,...,...,...,...
346,New York Times,European Central Bank Tweaks Strategy to Fight...,The central bank said it would tolerate inflat...,Neutral,"[central, bank, said, would, tolerate, inflati..."
347,New York Times,Ireland Fights for Its Tax Breaks,It and others are pushing back on a global agr...,Positive,"[others, pushing, back, global, agreement, mak..."
348,New York Times,A Digital Cat Is Melting Hearts (and Napping a...,The calico prances and dozes on a 26-by-62-foo...,Positive,"[calico, prances, dozes, byfoot, led, billboar..."
349,New York Times,The Parking Garage of the Future,Entrepreneurs envision a “mobility hub” that c...,Positive,"[entrepreneurs, envision, mobility, hub, save,..."


In [100]:
##converted list of words to new dataframe
f_col = ['source','words']
f_df = pd.DataFrame(columns = f_col)

w_df = final_df[['words']]
for i,r in w_df.iterrows():
    for word in r:
        for w in word:
            f_df = f_df.append({
                'source' : int(i),
                'words' : w
            },ignore_index=True)

In [101]:
f_df['source'] = f_df['source'].apply(lambda X : 'NDTV' if X<100 else 'New York Times' if X>189 else 'FOX')
f_df

,source,words
0,NDTV,amazoncom
1,NDTV,inc
2,NDTV,said
3,NDTV,online
4,NDTV,stores
...,...,...
5546,New York Times,long
5547,New York Times,work
5548,New York Times,hours
5549,New York Times,dwindling


In [102]:
f_df.groupby('source').count()

,words
source,
FOX,1672
NDTV,1771
New York Times,2108


In [88]:
cluster = MongoClient("mongodb_connection_string")
db = cluster["cluster_name"]
news = db['db_name_for_news']
words = db['db_name_fr_words']

if news.drop():
    records = json.loads(final_df.to_json())
    news.insert_one(records)
else:
    records = json.loads(final_df.to_json())
    news.insert_one(records)
    
if words.drop():
    records = json.loads(f_df.to_json())
    words.insert_one(records)
else:
    records = json.loads(f_df.to_json())
    words.insert_one(records)